In [117]:
import networkx as nx
import numpy as np
import scipy.optimize as sopt

Define the cost function $t(x)$ and its integration function, which is the objective function of UE.
list the time cost array of the flow from vertex 1 and 2 separately. 

In [118]:
def t(x):
    x13, x23, x14, x24, x34 = x
    t13 = 0.5+0.001*x13
    t23 = 0.5+0.001*x23
    t14 = 1.0+0.0005*x14
    t24 = 1.0+0.001*x24
    t34 = 1.0+0.002*x34
    t_array = [t13, t23, t14, t24, t34] 
    return t_array

def sum_tilde_t(x):
    x13, x23, x14, x24, x34 = x
    tilde_t13 = 0.5*x13+0.001*x13**2
    tilde_t23 = 0.5*x23+0.001*x23**2
    tilde_t14 = 1.0*x14+0.0005*x14**2
    tilde_t24 = 1.0*x24+0.001*x24**2
    tilde_t34 = 1.0*x34+0.002*x34**2
    return tilde_t13 + tilde_t23 + tilde_t14 + tilde_t24 + tilde_t34

 # calculate next point based on step size alpha, current point x and direction y
def cal_next_guess(alpha, x, y):
    next_point_x = x + alpha*(y-x)
    return next_point_x

Next we initialize an empty Graph object. 

In [119]:
guesses = [np.array([0, 0, 2000, 1000, 0])]
total_demand_1 = 2000
total_demand_2 = 1000

In [120]:
def create_graph(weight):
    # The following line initializes two empty directed graph objects
    G=nx.DiGraph()
    G.add_node(1, demand = -total_demand_1)  # source node 1
    G.add_node(2, demand = -total_demand_2)  # source node 2
    G.add_node(3, demand = 0)
    G.add_node(4, demand = total_demand_1+total_demand_2)    # sink node 4
    
    # t_array = [t13, t23, t14, t24, t34] 
    G.add_edge(1, 3, weight = weight[0])
    G.add_edge(2, 3, weight = weight[1])
    G.add_edge(1, 4, weight = weight[2])
    G.add_edge(2, 4, weight = weight[3])
    G.add_edge(3, 4, weight = weight[4])
    
    return G
    # print(G1.edges())
    # print(G1.nodes())
    # print(list(G1.nodes.data()))

Let us now add the arcs. As with nodes, there are multiple ways to add the arcs.

In [121]:
def print_graph_edge(G: nx.DiGraph):
    for node, nodedata in G.nodes.data():
        print('node:', node, 'demand:', nodedata['demand'])
        print('\tin_edge:')
        for in_edge in G.in_edges(node):
            weight = G.get_edge_data(in_edge[0],in_edge[1])['weight']
            cost = G.get_edge_data(in_edge[0],in_edge[1])['cost']
            print('\tedge:',in_edge, ' weight:', weight, 'cost:', cost)
        print('\tout_edge:')
        for out_edge in G.out_edges(node):
            weight = G.get_edge_data(out_edge[0],out_edge[1])['weight']
            cost = G.get_edge_data(out_edge[0],out_edge[1])['cost']
            print('\tedge:',out_edge, ' weight:', weight, 'cost:', cost)
    # print_graph_edge(G)

## Shortest Path

Let us now determine the shortest path using Djikstras algorithm. The function shown below returns a list which stores the shortest path from source to target.

In [122]:
# sp_source_1 = nx.dijkstra_path(G1,source = 1, target = 4, weight='cost')
# sp_source_2 = nx.dijkstra_path(G1,source = 2, target = 4, weight='cost')
# print('shortest path from source 1 to target 4:',sp_source_1)
# print('shortest path from source 2 to target 4:',sp_source_2)

In [123]:
def find_y(sp_source_1, sp_source_2, num_edg=5):
    y = np.array([0.0 for _ in range(num_edg)]) # x13, x23, x14, x24, x34
    if len(sp_source_1) == 2:   # 1-4
        y[2] += total_demand_1
    elif len(sp_source_1) == 3: # 1-3-4
        y[0] += total_demand_1
        y[4] += total_demand_1
    
    if len(sp_source_2) == 2:   # 2-4
        y[3] += total_demand_2
    elif len(sp_source_2) == 3: # 2-3-4
        y[1] += total_demand_2
        y[4] += total_demand_2
    
    return y

Function $step$: contain task for one iteration.
1. Define and calulate cost $t(x)$ and objective function $sum\_int\_t(x)$.
2. Find drection $y$: Set the flow direction $y$ to the route with minimum cost. 
3. Find the step size $\alpha \in (0,1)$ 
    1. Define the objective function after moving as $func$.
    2. Define the constraints $con$ to satisfy Total flow conservation and bound $\alpha \in (0,1)$.
    3. Use $scipy.optimize.minimize$ tool to obtain the value of step size $\alpha$.
5. Update flow as $x^{n+1} = x^n + \alpha * (y^n-x^n)$
6. Do convergence test. If satisfy the convergence test, stop the iteration; if not, enter next iteration. 


In [127]:
def step(i, guesses):
    print('in step',i+1)
    
    x = guesses[-1] # x13, x23, x14, x24, x34
    t_array = t(x)  # [t13, t23, t14, t24, t34] 
    
    # create a graph
    G = create_graph(weight=t_array)
    Z = sum_tilde_t(x)
    
    print('\tUpdate \t\tt\t', np.round(t_array,decimals=3), '\tZ(x) =', np.round(Z,decimals=2))
    
    # find y with min t in from source 1 and 2
    sp_source_1 = nx.dijkstra_path(G,source = 1, target = 4, weight='weight')
    sp_source_2 = nx.dijkstra_path(G,source = 2, target = 4, weight='weight')
    print('\tShortest path for source 1:', sp_source_1)
    print('\tShortest path for source 2:', sp_source_2)

    y = find_y(sp_source_1, sp_source_2, num_edg=len(x))
    print('\tDirection \ty\t', np.round(y,decimals=2))

    # objective function to minimize
    def func(alpha):
        next_point_x = cal_next_guess(alpha,x,y)
        return sum_tilde_t(next_point_x)
    
    # find alpha that minimize objective function
    alpha = np.array([0.0])
    bound = [(0,1)]
    res = sopt.minimize(func, alpha, bounds=bound)
    alpha = res.x
    if res.success == False:
        print('\tOptimization False')
        print('\t',res.message)
        return False
        
    print('\tStep \t\talpha\t', np.round(alpha,decimals=3))

    # update x
    next_guess = cal_next_guess(alpha,x,y)
    guesses.append(next_guess)
    print('\tMove \t\tx\t', np.round(next_guess,decimals=3))
    return True
    

In [128]:
if __name__ == '__main__':
    # x13, x23, x14, x24, x34 = x
    guesses = [np.array([0, 0, 2000, 1000, 0])]
    print('Init Guesses\t\tx\t',guesses[0])

    for i in range(4):
        res = step(i, guesses)
        if res == False:
            break
    

Init Guesses		x	 [   0    0 2000 1000    0]
in step 1
	Update 		t	 [0.5 0.5 2.  2.  1. ] 	Z(x)= 6000.0
	Shortest path for source 1: [1, 3, 4]
	Shortest path for source 2: [2, 3, 4]
	Direction 	y	 [2000. 1000.    0.    0. 3000.]
	Step 		alpha	 [0.087]
	Move 		x	 [ 173.077   86.538 1826.923  913.462  259.615]
in step 2
	Update 		t	 [0.673 0.587 1.913 1.913 1.519] 	Z(x)= 5805.29
	Shortest path for source 1: [1, 4]
	Shortest path for source 2: [2, 4]
	Direction 	y	 [   0.    0. 2000. 1000.    0.]
	Step 		alpha	 [0.]
	Move 		x	 [ 173.077   86.538 1826.923  913.462  259.615]
in step 3
	Update 		t	 [0.673 0.587 1.913 1.913 1.519] 	Z(x)= 5805.29
	Shortest path for source 1: [1, 4]
	Shortest path for source 2: [2, 4]
	Direction 	y	 [   0.    0. 2000. 1000.    0.]
	Step 		alpha	 [0.]
	Move 		x	 [ 173.077   86.538 1826.923  913.462  259.615]
in step 4
	Update 		t	 [0.673 0.587 1.913 1.913 1.519] 	Z(x)= 5805.29
	Shortest path for source 1: [1, 4]
	Shortest path for source 2: [2, 4]
	Direction 	y	 